#  一. 预算决定波系分类

If we consider all the elastic and plastic waves, there are 4*4 sixteen possible cases. For simple, we only  consider the cases in the left side and samilar process will be taken in the right.
<img src="Case1.png" width = "350" height = "350" div align=center />
<center> Case 1
<img src="Case2.png" width = "350" height = "350" div align=center />
    Case 2
    <img src="Case3.png" width = "350" height = "350" div align=center />
    Case 3
<img src="Case4.png" width = "350" height = "350" div align=center />
    Case 4


## 通过HLLCE进行预算
考虑双弹性波情况
<img src="Case5.png" width = "350" height = "350" div align=center />
 <center>   With only plastic or plastic shock wave

By using the relation of $u_L^* =u_R^* = s^*$, the speed of contact wave can be evaluated as
\begin{equation}
 \hat{s}^* = \frac{\sigma_L-\sigma_R+\rho_L u_L(s_L-u_L)-\rho_R u_R(s_R-u_R)}{\rho_L(s_L-u_L)-\rho_R(s_R-u_R)},
\end{equation}
the density is solved as
\begin{equation}\label{eq:rhoLs}
  \hat{\rho}_L^* = \frac{\rho_L(u_L-s_L)}{\hat{s}^*-s_L}.
\end{equation}
Similarly, the density behind the right going wave can be given as
\begin{equation}
  \hat{\rho}_R^* = \frac{\rho_R(u_R-s_R)}{\hat{s}^*-s_R}.
\end{equation}



The deviatoric stress is evaluated as
\begin{equation}  \label{sxx1}
  \hat{s}_{xxL}^*=-\frac{4}{3}\mu\text{ln}(\frac{\hat{\rho}_L^*}{\rho_L})+s_{xxL}, \quad   \hat{s}_{xxR}^*=-\frac{4}{3}\mu\text{ln}(\frac{\hat{\rho}_R^*}{\rho_R})+s_{xxR}.
\end{equation}

If the speeds of left and right going waves are given, we can evaluate all states in the star regions on the two sides of  the contact wave. Here we define the speeds of left and right going waves as
    \begin{equation}
      s_L = \text{min} (u_L-c_L, u_R-c_R, 0),  \quad s_R = \text{max}(u_L+c_L, u_R+c_R, 0).
    \end{equation}


In [104]:
function Presolve(varL::Var, varR::Var, conL::Const,conR::Const)
        
    ρL, uL, pL, sxxL = varL.ρ, varL.u, varL.p, varL.sxx
    ρR, uR, pR, sxxR = varR.ρ, varR.u, varR.p, varR.sxx
    σL  = -pL+sxxL
    
    UL = [ρL, uL, pL, sxxL]
    UR = [ρR, uR, pR, sxxR]
    cL = sound(UL[:],conL)
   # eL = PToe(ρL, pL, conL)
    
    σR  = -pR+sxxR
    cR = sound(UR[:],conR)
  #  eR = PToe(ρR, pR, conR)
    
    sL = min(uL-cL, uR-cR)
    sR = max(uL+cL, uR+cR)
        
    s_star = (σL - σR + ρL*uL*(sL-uL) - ρR* uR*(sR-uR))/(ρL*(sL-uL)-ρR*(sR-uR))
        
    ρLstar = ρL*(uL-sL)/(s_star - sL)
    ρRstar = ρR*(uR-sR)/(s_star - sR)
        
    sxxLstar =  sxxL -4/3*conL.μ *(log(ρLstar) - log(ρL))
    sxxRstar =  sxxR -4/3*conR.μ *(log(ρRstar) - log(ρR))
 #   @show sxxLstar, sxxRstar,ρLstar,ρRstar
    caseL = CaseSelect(sxxL,sxxLstar, ρL, ρLstar,conL,1)
    caseR = CaseSelect(sxxR,sxxRstar, ρR, ρRstar,conR,2)
    σLstar = σL -ρL*(sL-uL)*(s_star-uL)
    pLstar =sxxLstar - σLstar
  #  @show pLstar,sxxLstar,ρLstar, s_star
    
 #   return caseL, caseR
    return ρLstar, ρRstar, caseL,caseR
end

Presolve (generic function with 1 method)

## 分类
Also consider the cases in the left, 
### Case 1
The deviatoric stress satisfies 
$$| \hat{s}_{xxL}^* | \ge  \frac{2}{3}Y_0   \quad \text{and} \quad | s_{xxL} | \ge  \frac{2}{3}Y_0 \quad  \\\text{or} \quad | \hat{s}_{xxL}^* | \lt  \frac{2}{3}Y_0   \quad \text{and} \quad | s_{xxL} | \lt  \frac{2}{3}Y_0 $$ 

The density satisfies
    $$ \hat{\rho}_L^* < \rho_L$$


### Case 2
The deviatoric stress satisfies 
$$| \hat{s}_{xxL}^* | \ge  \frac{2}{3}Y_0   \quad \text{and} \quad | s_{xxL} | \lt  \frac{2}{3}Y_0  $$ 

The density satisfies
    $$ \hat{\rho}_L^* < \rho_L$$

### Case 3
The deviatoric stress satisfies 
$$| \hat{s}_{xxL}^* | \ge  \frac{2}{3}Y_0   \quad \text{and} \quad | s_{xxL} | \ge  \frac{2}{3}Y_0 \quad  \\\text{or} \quad | \hat{s}_{xxL}^* | \lt  \frac{2}{3}Y_0   \quad \text{and} \quad | s_{xxL} | \lt  \frac{2}{3}Y_0 $$ 
The density satisfies
    $$ \hat{\rho}_L^* > \rho_L$$


### Case 4
The deviatoric stress satisfies 
$$| \hat{s}_{xxL}^* | \ge  \frac{2}{3}Y_0   \quad \text{and} \quad | s_{xxL} | \lt  \frac{2}{3}Y_0  $$ 

The density satisfies
    $$ \hat{\rho}_L^* > \rho_L$$

In [21]:
function CaseSelect(sxx₀::Float64, sxx₁::Float64,ρ₀::Float64, ρ₁::Float64,con::Const,LoR::Int)
    Y0 = con.Y0
      if abs(sxx₁) ≥ 2/3*Y0  && abs(sxx₀) < 2/3*Y0   
        if ρ₁ > ρ₀
            if LoR == 1
                case = Shock(2) # 左侧双激波
            else
                case = Shock(4) # 右侧双激波 
            end
        else
            if LoR == 1
                case = Rare(2) # case2 Left
            else
                case = Rare(4) # case2 Right
            end
        end
    else
        if ρ₁ > ρ₀
            if LoR == 1
                case = Shock(1) # case3
                else
                case = Shock(3)
            end
        else
            if LoR == 1
                case = Rare(1) #case1 Left
            else
                case = Rare(3)
            end
        end
    end
    return case
end

CaseSelect (generic function with 1 method)

## 1 左侧膨胀波

<img src="Case1.png" width = "500" height = "500" div align=center />
如图，根据膨胀波关系式有

\begin{equation}
    du+\frac{c}{\rho} d\rho= 0\\
    d\sigma +c^2 d\rho = 0
\end{equation}
 where $\sigma = -p +s_{xx}$, and 
 \begin{equation}
    dp-S^2d\rho = 0\\
    ds_{xx}+\frac{4\mu}{3\rho} d\rho= 0 
    \end{equation}
Integration from $Q_L$ to $Q^*_L$
\begin{equation}
u_L^* =- \int_{\rho_L}^{\rho^*_L}\frac{c}{\rho}d\rho + u_L\\
\sigma_L^* = -\int_{\rho_L}^{\rho^*_L} c^2 d\rho +\sigma_L
\end{equation}
and 
\begin{equation}
p_L^* = \int_{\rho_L}^{\rho^*_L}S^2 d\rho + p_L\\
s_{xxL}^* = -\int_{\rho_L}^{\rho^*_L} \frac{4\mu}{3\rho} d\rho +s_{xxL}
\end{equation}


For right side
\begin{equation}
u_R^* = \int_{\rho_R}^{\rho^*_R}\frac{c}{\rho}d\rho + u_R\\
\sigma_R^* = -\int_{\rho_R}^{\rho^*_R} c^2 d\rho +\sigma_R
\end{equation}
and 
\begin{equation}
p_R^* = \int_{\rho_R}^{\rho^*_R}S^2 d\rho + p_R\\
s_{xxR}^* = -\int_{\rho_R}^{\rho^*_R} \frac{4\mu}{3\rho} d\rho +s_{xxR}
\end{equation}

In [22]:
struct Shock
    cs::Int
end

struct Rare
    cs::Int
    
end
##case 说明  cs=1 为左单波 cs=2 为左双波
##          cs=3 为右单波 cs= 4 为右双波

$s_{xxL}^*$ can be given directly as
$$s_{xx}(\rho) = s_{xxL} - \frac{4}{3}\mu [\text{ln}(\rho) - \text{ln}(\rho_L)] $$


In [75]:
function Sxx(ρ, var0::Var, con::Const)
    sxx0,ρ0 = var0.sxx,var0.ρ
    Y0   = con.Y0
    
    sxx = sxx0 -4/3*con.μ *(log(ρ) - log(ρ0))
    if abs(sxx) ≥ 2/3*Y0
        sxx = 2/3*Y0*sign(sxx)
    end
    return sxx
end

Sxx (generic function with 1 method)

Then we evaluate $p_L^*$, as
$$S^2 = a^2 - \rho_0 \Gamma_0 \frac{s_{xx}}{\rho^2} = a_0^2\frac{\partial f}{\partial \eta}+\frac{p}{\rho^2}\rho_0\Gamma_0-\rho_0 \Gamma_0\frac{s_{xx}}{\rho^2}$$


So 
$p(\rho)$ is in the form of 
$$ \tag{2.1} p(\rho) = \int_{\rho_L}^{\rho} \lambda_1 \frac{p}{\rho^2}+ f_2(\rho) d\rho + p_L$$
where 
$$ \lambda_1 = \rho_0 \Gamma_0 \quad f_2(\rho) = a_0^2\frac{\partial f}{\partial \eta}- \lambda_1\frac{s_{xx}(\rho)}{\rho^2}$$
it can be written in the different form as


$$p'(\rho) - \lambda_1 \frac{p}{\rho^2} = f_2(\rho), \quad p(\rho_L) = p_L$$ 
it can be solved out as 
$$ p(\rho) = p_Le^{\frac{\lambda_1}{\rho_L}-\frac{\lambda_1}{\rho}} +e^{-\frac{\lambda_1}{\rho}}\int_{\rho_L}^\rho f_2(x) e^{\frac{\lambda_1}{x}}dx$$

In [24]:
function fp(ρ::Float64, var0::Var, con::Const, case::Rare)
    ρ₀,p₀ = var0.ρ,var0.p
    ρ0,Γ0,a0= con.ρ0,con.Γ0,con.a0   
    λ₁ = ρ0*Γ0
    f2(ρ) =a0^2*fηη(ρ, con) -λ₁*Sxx(ρ, var0,con)/ρ^2

    p = p₀*exp(λ₁/ρ₀ - λ₁/ρ) + exp(-λ₁/ρ)*GaussIntegral(ρ->
       f2(ρ)*exp(λ₁/ρ), ρ₀, ρ,5)
    return p
end
#pLStar = fp(ρLStar,varL,con)

fp (generic function with 1 method)

According to $\sigma = -p +s_{xx}$ we can get $\sigma(\rho)$.
$$ g_L(\rho^*_L) =\sigma_L^*= -p(\rho_L^*)+s_{xx}(\rho_L^*)$$
As $c^2 = S^2 +\frac{4\mu}{3\rho}$, 
$$ u_L^* = u_L - \int_{\rho_L}^{\rho^*_L} 
\frac{c}{\rho}d\rho$$
can be solve by the numerical integral, and 
$$ f_L(\rho^*_L) = u_L^*$$

In [25]:
function fu(ρ::Float64, var0::Var, con::Const,case::Rare)
    ρ₀,u₀ = var0.ρ, var0.u
    a0,ρ0,Γ0,μ= con.a0, con.ρ0, con.Γ0, con.μ
    λ₁ = ρ0*Γ0
    S2(ρ) = a0^2*fηη(ρ,con)+λ₁*(fp(ρ,var0,con,case)-Sxx(ρ,var0,con))/ρ^2
    function f2(ρ)
        if case.cs == 1 &&  case.cs ==2
            return -√(S2(ρ)+4μ/(3ρ)) /ρ
        else
            return √(S2(ρ)+4μ/(3ρ)) /ρ
        end
    end
    
    u = u₀ + GaussIntegral(ρ->f2(ρ), ρ₀, ρ, 5)
    return u
end

fu (generic function with 1 method)

### 数值积分
We use the Gaussian quadradure to intergrate the integrations above.

For a function $f(x)$,  the intergration from $-1$ to $1$ is given as
$$ 
  \int_{-1}^1 f(x)dx \approx \sum_{i=0}^n \omega_i f(p_i)
$$
$\omega_i$ is the weight, and $p_i$ is the integrating point.
For different orders they are listed as

|number of points,$n$| Points, $p_i$| Weights, $\omega_i$|
|:-----|-----:|:-----:|
|1     |0     |2      |
|2     |$\pm \frac{1}{\sqrt{3}}$| 1|
|3     |0, $\pm \sqrt{\frac{3}{5}}$| $\frac{8}{9}$,$\frac{5}{9}$, $\frac{5}{9}$|

For a integral over $[x_0,x_1]$, this change of interval can be done as 
 $$ \int_{x_0}^{x_1}f(x) dx= \frac{x_1-x_0}{2}\int_{-1}^{1} f(\frac{x_1-x_0}{2}x+\frac{x_0+x_1}{2})dx
 $$

In [26]:
function GaussIntegral(f::Function,x₀::Float64,x₁::Float64,order::Int)
    t₁= (x₁-x₀)/2
    t₂= (x₁+x₀)/2
    ω = zeros(Float64, 5)
    p = zeros(Float64, 5)
    
    if order == 1
        ω[1] = 2.0
        p[1] = 0.0
    elseif order == 3
        ω[1] = 1.0; ω[2] = 1.0
        p[1] = 1/√3.0; p[2] = -1/√3.0
    elseif order == 5
        ω[1] = 8.0/9; ω[2] = 5.0/9; ω[3] = 5.0/9
        p[1] = 0.0; p[2] = -√(3.0/5); p[3] = √(3.0/5)
    elseif order == 7
        ω[1] = (18+√30)/36; ω[2] = (18+√30)/36
        ω[3] = (18-√30)/36; ω[4] = (18-√30)/36
        p[1] = √(3/7-2/7*√(6/5)); p[1] = -√(3/7-2/7*√(6/5))
        p[3] = √(3/7+2/7*√(6/5)); p[4] = -√(3/7+2/7*√(6/5))
    end
    ∑ =sum( t₁*ω[i]*f(t₁*p[i]+t₂) for i in 1:floor(Int,order/2)+1)

    return ∑
end

GaussIntegral (generic function with 1 method)

## 2 左侧双膨胀波
<img src="Case2.png" width = "500" height = "500" div align=center />

### 弹性稀疏波后状态
 In the region $\widetilde{Q}_L$
$$
  \widetilde{s}_{xxL} =\frac{2}{3}Y_0
$$
and the density is
$$ \widetilde{\rho}_L = \rho_L \text{exp}\left(-\frac{Y_0}{2\mu}+\frac{3 s_{xxL}}{4\mu}\right)$$


According to ($2.1$) we can solve the pressure $\widetilde{p}_L$ ,then the velocity $\widetilde{u}_L$  is solved out.

In [27]:
function Ṽar(var0::Var, case, con::Const)

    Y0, μ= con.Y0, con.μ
    sxx,ρ = var0.sxx,var0.ρ

    cs = case.cs
    if cs == 1 || cs == 3
        var1 = var0
    elseif cs == 2 || cs == 4
        
        if typeof(case) == Rare 
            sxx1 = 2/3*Y0
            ρ1 = ρ*exp(-Y0/(2μ)+(3sxx)/(4μ))
        elseif typeof(case) == Shock
            sxx1 = -2/3*Y0
            ρ1 = ρ*exp(Y0/(2μ)+(3sxx)/(4μ))
        end
        p1 = fp(ρ, var0, con, case)
        u1 = fu(ρ, var0, con, case)
    
        var1= Var(ρ1, u1, p1, sxx1)
    end
    return var1
end


Ṽar (generic function with 1 method)

### From state $\widetilde{Q}_L$  to state $Q_L^*$ 

The process is same to Case 1. 

## 3 左侧激波

<img src="Case3.png" width = "500" height = "500" div align=center />
According to the ralation of Runkine-Hugoniot,

\begin{equation} \tag{1.2}
F_L^*=F_L+s_L(U_L^*-U_L)
  \end{equation}


As is
\begin{align}
    &\rho^*_L(u^*_L-s_L) = \rho_L(u_L-s_L), \tag{1.3}\\
    &\rho^*_Lu^*_L(u^*_L-s_L) = \rho_Lu_L(u_L-s_L)+\sigma^*_L-\sigma_L,  \tag{1.4}\\
    &\rho^*_LE^*_L(u^*_L-s_L) = \rho_LE_L(u_L-s_L)+\sigma^*_L u^*_L-\sigma_Lu_L, \tag{1.5}\\
\end{align}


According to ($1.3$) we have 
\begin{equation}\tag{1.6}
s_L = \frac{\rho_L^* u_L^* - \rho_L u_L}{\rho_L^*-\rho_L}
\end{equation}

Subtituting ($1.3$) into ($1.4$) yields
\begin{equation}\tag{1.7}
\rho_L(u^*_L-u_L)(u_L-s_L) = \sigma_L^*-\sigma_L
\end{equation}

Also according to ($1.3$), we have 
\begin{equation}
u_L-s_L = \frac{(u_L - u_L^*)\rho_L^*}{\rho_L^* -\rho_L}
\end{equation}
Then subtituting it into ($1.7$)
\begin{equation}\tag{1.8}
  -t(u^*_L-u_L)^2 = \sigma^*_L-\sigma_L,
\end{equation}
where $ t=\frac{\rho_L \rho^*_L}{\rho^*_L-\rho_L}$.

Similar to (1.7), (1.5) can be changed into
$$ \tag{1.9} t(u_L-u^*_L)(E^*_L-E_L) =\sigma^*_L u^*_L-\sigma_Lu_L
$$


Because $E = e+\frac{1}{2}u^2$, we can get 
$$ \tag{1.10}
e_L^* - e_L = - \frac{\sigma_L +\sigma_L^*}{2t}
$$
Also we use the EOS of Mie-Gr\"uniesen  
$$\tag{1.11}
  p(\rho,e) = \rho_0 a_0^2f(\eta)+ \rho_0 \Gamma_0 e,
$$
can get
$$ \tag{1.12}
  e=c_0 p-c_1f(\rho/\rho_0),
$$
where $c_0=\frac{1}{\rho_0\Gamma_0}$ and $c_1=\frac{a_0^2}{\Gamma_0}$.

Using (1.12), from (1.10) we can get 
 $$ \tag{1.13}
 p^*_L= \frac{2t(c_1f(\rho^*_L)+e_L)-(\sigma_L+s^*_{xxL})}{2tc_0-1},
$$

And we use the relation of $s_{xxL}$ and $\rho$
$$  s_{xxL}^*=s_{xxL}^*(\rho_L^*)=-\frac{4}{3}\mu\text{ln}(\frac{\rho^*_{L}}{\rho_{L}})+s_{xxL}
$$
we can get
$$ \tag{1.14} \sigma_L^*  = \frac{2t[c_1f(\rho^*_L)-c_0 s_{xxL}^*(\rho^*_L)+e_L] -\sigma_L}{1-2tc_0}$$

In [77]:
function fp(ρ::Float64, var0::Var, con::Const,case::Shock)
   
    ρ₀,p₀,sxx₀ = var0.ρ,var0.p,var0.sxx
    ρ0,Γ0,a0 = con.ρ0,con.Γ0,con.a0
    
    σ₀ = -p₀+sxx₀
    e₀ =  PToe(ρ₀, p₀, con)
   # @show e₀
    t =  (ρ-ρ₀)/(ρ₀*ρ) ; c₀ = 1/(Γ0*ρ0); c₁= a0^2/Γ0
  
    p = (2(c₁*fη(ρ,con) + e₀) -t*(σ₀+Sxx(ρ,var0,con)))/(-t+2c₀)
 
    
 #   @show t, (2c₀-t),c₀
    return p
end

fp (generic function with 2 methods)

Using ($1.8$) we have 
$$ \tag{1.15} 
u_L^* = u_L - \sqrt{\frac{\sigma_L-\sigma_L^*}{t}} $$


In [29]:
function fu(ρ::Float64, var0::Var, con::Const,case::Shock)
   
    ρ₀,u₀,p₀,sxx₀ = var0.ρ,var0.u,var0.p,var0.sxx
    σ₀ = -p₀+sxx₀
    
    sxx=Sxx(ρ,var0,con)
    p  = fp(ρ, var0, con, case)
    σ = -p+sxx
    
    t =  (ρ-ρ₀)/ρ₀*ρ
    if case.cs == 1 || case.cs == 2
        u = u₀ - √(t*(σ₀ - σ))
    
    else
        u = u₀ + √(t*(σ₀ - σ))
    end
#    @show t, σ₀ - σ
    return u
end

fu (generic function with 2 methods)

## 4 左侧双激波
<img src="Case4.png" width = "500" height = "500" div align=center />

### 弹性激波后状态
 In the region $\widetilde{Q}_L$
$$
  \widetilde{s}_{xxL} =\frac{2}{3}Y_0
$$
and the density is
$$ \widetilde{\rho}_L = \rho_L \text{exp}\left(\frac{Y_0}{2\mu}+\frac{3 s_{xxL}}{4\mu}\right)$$
From ($1.14$) and ($1.15$)  other viriables can be get.
Then use the same process from $\widetilde{Q}_L$ to $Q_L^*$ as Case 3. 

## 求解$\sigma_L^*$ 和$ u_L^*$
Let $f_{L}(\rho^*_L) = u_L^* $ and $g_{L}(\rho^*_L) = \sigma_L^*$, samilarly, we let
$f_{R}(\rho^*_R) = u_R^*$ and $g_{R}(\rho^*_R) = \sigma_R^*$, we have the relation
\begin{equation}\tag{1.1}
F(\rho^*_L,\rho^*_R) = f_L(\rho^*_L) - f_R(\rho^*_R) = 0\\
G(\rho^*_L,\rho^*_R) = g_L(\rho^*_L) - g_R(\rho^*_R) = 0
\end{equation}

In [30]:
    fL(ρ) = fu(ρ, var0L, conL, caseL)
    fR(ρ) = fu(ρ, var0R, conR, caseR)
    gL(ρ) = -fp(ρ,var0L,conL,caseL)+Sxx( ρ, var0L, conL)
    gR(ρ) = -fp(ρ,var0R,conR,caseR)+Sxx( ρ, var0R, conR)

    fL¹(ρ) = Derivative(fL,ρ)
    fR¹(ρ) = Derivative(fR,ρ)
    gL¹(ρ) = Derivative(gL,ρ)
    gR¹(ρ) = Derivative(gR,ρ)

gR¹ (generic function with 2 methods)


###  牛顿迭代
We use the newton iteration method to evaluate $\rho^*_L$ and $\rho^*_R$.
\begin{equation}
\left[ \begin{array}{l}
 \rho _{R,(k+1)}^*\\
\rho_{L,(k+1)}^*\\
\end{array}
\right] = 
\left[ \begin{array}{l}
 \rho _{R,(k)}^*\\
\rho_{L,(k)}^*\\
\end{array}
\right]-
\left[ \begin{array}{ll}
\frac{\partial F_{(k)}}{\partial \rho_R^*} & \frac{\partial F_{(k)}}{\partial \rho_L^*}\\
\frac{\partial G_{(k)}}{\partial \rho_R^*} & \frac{\partial G_{(k)}}{\partial \rho_L^*}\\
\end{array}
\right]^{-1}
\left[ \begin{array}{l}
F_{(k)}\\
G_{(k)}\\
\end{array}
\right]
\end{equation}


from Eq.($1.1$) we can get
\begin{equation}
\left[ \begin{array}{l}
 \rho _{R,(k+1)}^*\\
\rho_{L,(k+1)}^*\\
\end{array}
\right] = 
\left[ \begin{array}{l}
 \rho _{R,(k)}^*\\
\rho_{L,(k)}^*\\
\end{array}
\right]-
\left[ \begin{array}{ll}
-\frac{\partial f_{R,(k)}}{\partial \rho_R^*} & \frac{\partial f_{L,(k)}}{\partial \rho_L^*}\\
-\frac{\partial g_{R,(k)}}{\partial \rho_R^*} & \frac{\partial g_{,L(k)}}{\partial \rho_L^*}\\
\end{array}
\right]^{-1}
\left[ \begin{array}{l}
F_{(k)}\\
G_{(k)}\\
\end{array}
\right]
\end{equation}
where $\rho_{L,(k)}^*$ and $\rho_{R,(k)}^*$ are the k-th iteration values, $F(k) = F(\rho^*_{L,(k)},\rho^*_{R,(k)})$,$G(k) = G(\rho^*_{L,(k)},\rho^*_{R,(k)})$.


$$u = u - J^{-1}F$$


For a rarefaction wave in the left, the derivatives are solved by
\begin{equation}
\frac{\partial f_{L,(k)}}{\partial \rho^*_{L,(k)}} = \frac{c(\rho^*_{L,(k)})}{\rho^*_{L,(k)}}\\
\frac{\partial g_{L,(k)}}{\partial \rho^*_{L,(k)}} = c(\rho^*_{L,(k)})^2
\end{equation}

#### 初始条件

We take an intial iteration value as 
\begin{equation}
\rho_{L,(0)}^* = \rho_{R,(0)}^* = \frac{\rho_L +\rho_R}{2}
\end{equation}

In [31]:
function Derivative(f::Function,x::Float64)
    ϵ₀ = 1e-8
    if abs(x) >= ϵ₀
        ϵ= ϵ₀*x
    else
        ϵ =ϵ₀
    end
    f¹(x) = (f(x+ϵ)-f(x))/ϵ
    return f¹(x)
end

Derivative (generic function with 1 method)

In [32]:
function NewtonIter!(U::Array{Float64,1},J::Array{Float64,2},F::Array{Float64,1}, TOL::Float64 = 1.e-6) 
    I, = size(U)
    U_new = zeros(Float64, I)
    while true  
        c = max(abs(F[1]),abs(F[2]))
        if c <= TOL
            break
        end
        U_new = U - inv(J)*F # same to inv(J)×F
   #     c= CHA(U,U_new,F)
     #   if c <= TOL
      #      break
      #  end
    end
    return U
    
end   

NewtonIter! (generic function with 2 methods)

#### 收敛准则

The iteration procedure stops with the criterion of
$\text{CHA}\le \text{TOL}$, where 
\begin{equation}
\text{CHA} = \text{max} \left[ 
\frac{2|(\rho_L^*)_{(k+1)}-(\rho_L^*)_{(k)}|}{|(\rho_L^*)_{(k+1)}+(\rho_L^*)_{(k)}|}, \frac{2|(\rho_R^*)_{(k+1)}-(\rho_R^*)_{(k)}|}{|(\rho_R^*)_{(k+1)}+(\rho_R^*)_{(k)}|}, |F_{(k)}|, |G_{(k)}| \right]
\end{equation}
and TOL is a small tolerance by numerical experiments, we take as $\text{TOL} = 10^{-6}$.

In [33]:
function CHA(R0::Array{Float64,1}, R1::Array{Float64,1}, F::Array{Float64,1})
    
    ρₗ₀ = R0[1]; ρᵣ₀ = R0[2]
    ρₗ₁ = R1[1]; ρᵣ₁ = R1[2]
    
    CHA = max(abs(2(ρₗ₁-ρₗ₀)/(ρₗ₁+ρₗ₀)),
              abs(2(ρᵣ₁-ρᵣ₀)/(ρᵣ₁+ρᵣ₀)), F[1], F[2])
    return CHA
end   

CHA (generic function with 1 method)

In [34]:
#include("/home/bfly/workspace/Juliastudy/Lagrangian_1d-2materials.jl")

Plastic (generic function with 1 method)

In [35]:
    fL(ρ, varL, conL, caseL) = fu(ρ, varL, conL, caseL)
    fR(ρ, varR, conR, caseR) = fu(ρ, varR, conR, caseR)
    gL(ρ, varL, conL, caseL) = -fp(ρ,varL,conL,caseL)+Sxx( ρ, varL, conL)
    gR(ρ, varR, conR, caseR) = -fp(ρ,varR,conR,caseR)+Sxx( ρ, varR, conR)

    fL¹(ρ, varL, conL, caseL) = Derivative(ρ->fL(ρ, varL, conL, caseL),ρ)
    fR¹(ρ, varR, conR, caseR) = Derivative(ρ->fR(ρ, varR, conR, caseR),ρ)
    gL¹(ρ, varL, conL, caseL) = Derivative(ρ->gL(ρ, varL, conL, caseL),ρ)
    gR¹(ρ, varR, conR, caseR) = Derivative(ρ->gR(ρ, varR, conR, caseR),ρ)
 

gR¹ (generic function with 2 methods)

# 根据迭代结果重新求解左右状态


In [105]:
function Exact_Riemann(uL::Array{Float64,1}, uR::Array{Float64,1}, conL::Const, conR::Const )
        
    
    var0L = Var(uL[1],uL[2],uL[3],uL[4])
    var0R = Var(uR[1],uR[2],uR[3],uR[4])
 #  @show var0L
    
###预求解和分类    
    ρLInit, ρRInit,caseL,caseR = Presolve(var0L, var0R, conL, conR)
  # println(ρLInit,ρRInit)
###求解  Q̂ 
    @show caseL,caseR
    varL = Ṽar(var0L, caseL,conL)
    varR = Ṽar(var0R, caseR,conR)
    ρL, uL, pL, sxxL = varL.ρ, varL.u, varL.p, varL.sxx
    ρR, uR, pR, sxxR = varR.ρ, varR.u, varR.p, varR.sxx

   # @show varL, varR
    ###迭代初始化    
    ρLStar = ρLInit
    ρRStar = ρRInit
   
    
    TOL = 2.e-5
    i = 1
    U = zeros(Float64, 2)
    
    println(ρLStar,"  ",ρRStar)
    while i < 30
        i += 1

        J = [[-fR¹(ρRStar,varR, conR, caseR) fL¹(ρLStar,varL, conL, caseL)]; 
             [-gR¹(ρRStar,varR, conR, caseR) gL¹(ρLStar,varL, conL, caseL)]]
    
        U = [ρRStar, ρLStar]
    
        F = [fL(ρLStar,varL, conL, caseL)- fR(ρRStar,varR, conR, caseR), 
             gL(ρLStar,varL, conL, caseL)- gR(ρRStar,varR, conR, caseR)]
      
    
      
     c = max(abs(F[1]),abs(F[2])) 
    
        @show U, F,J
    
      if c <= TOL
            break
        end
    #   @show F
        U = U - J\F
        ρRStar = U[1]
        ρLStar = U[2]
   end
   
    
    ### 重新求解
    pLStar = fp(ρLStar, varL, conL,caseL)
    s_Star = fu(ρLStar, varL, conL,caseL)
    sxxLStar = Sxx(ρLStar, varL, conL)

    pRStar = fp(ρRStar, varR, conR,caseR)
    sxxRStar = Sxx(ρRStar, varR, conR)
    FL=zeros(Float64, 4)
    FR=zeros(Float64, 4)
    FL[1] = 0.0
    FL[2] = pLStar-sxxLStar
    FL[3] = (pLStar-sxxLStar)*s_Star
    FL[4] = -4conL.μ/3*s_Star
    uuh = s_Star

    FR[1] = 0.0
    FR[2] = pRStar-sxxRStar
    FR[3] = (pRStar-sxxRStar)*s_Star
    FR[4] = -4conR.μ/3*s_Star
    uuh = s_Star
    
    return FL,FR, uuh
end

Exact_Riemann (generic function with 1 method)

In [134]:
uL = [2800, 300.0, 1.e6,0.0]
uR = [2600, 0.0, 1.e2,0.0]

conL = Const(3e8,2785,2.0,2.76e10,5328.0,1.338)
conR = Const(3e8,2785,2.0,2.76e10,5328.0,1.338)


Const(3.0e8, 2785.0, 2.0, 2.76e10, 5328.0, 1.338)

In [135]:
Exact_Riemann(uL,uR,conL,conR)

(caseL, caseR) = (Shock(2), Shock(4))
2862.9391102942  2660.6726029215815
(U, F, J) = ([2660.67, 2862.94], [-5.06291e5, -1.6047e8], [-5375.63 -5625.47; 2.79036e7 -3.06082e7])
(U, F, J) = ([2615.28, 2816.32], [-11092.3, -2.34192e6], [-5134.43 -5385.84; 2.634e7 -2.89855e7])
(U, F, J) = ([2614.22, 2815.27], [-5.76377, 373.247], [-5128.83 -5380.51; 2.63039e7 -2.89497e7])
(U, F, J) = ([2614.22, 2815.27], [-1.65635e-6, 0.00102934], [-5128.83 -5380.5; 2.63039e7 -2.89496e7])
(U, F, J) = ([2614.22, 2815.27], [4.83709e-10, -2.18749e-5], [-5128.83 -5380.5; 2.63039e7 -2.89496e7])
(U, F, J) = ([2614.22, 2815.27], [5.52291e-10, 1.29342e-5], [-5128.83 -5380.5; 2.63039e7 -2.89496e7])


([0.0, 2.01276e8, 5.00656e10, -9.15368e12], [0.0, 2.01276e8, 5.00656e10, -9.15368e12], 248.74121296705704)